This file looks at producing example images

Let's try to sharpen the images or look at the surrounding area of each pixel to highlight areas that may have an issue. Then let's look at
trying to quantify the noise in each image.

In [1]:
# Reading the data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from copy import copy
from matplotlib import cm, colors
import cv2

objectwidth = 83
objectheight = 122
xspacing = 133
yspacing = 270
xstart = 293
ystart = 268
xend = 1730
yend = 1770
powderthickness = 80
endlayer = 187

paths = pathlib.Path('./OT data 80 um/int').glob('*.tif')
paths_sorted = [x for x in paths]
paths_sorted.sort()
integrals = np.array([np.array(plt.imread(path)) for path in paths_sorted])[0:endlayer]

objectinfo = pd.read_csv('Parameters.csv', names=["Object", "P", "S", "H", "Porosity", "Label"])
objectinfo.insert(1, "VED", objectinfo.P * 1000/(objectinfo.S * objectinfo.H * powderthickness))
objectCoordinates = [[x, x+objectwidth, y, y+objectheight] for y in reversed(range(
    ystart, yend, objectheight + yspacing)) for x in range(xstart, xend, xspacing + objectwidth)]
coorddf = pd.DataFrame(objectCoordinates, columns=['xstart', 'xend', 'ystart', 'yend'])
objectinfo = coorddf.join(objectinfo)

del coorddf
del objectCoordinates
del paths_sorted

Lets sharpen the image and then filter out all background pixels and set them to nan

In [ ]:
# Tweaking parameter settings
emptyRatio = 47
sharpening = ['direct', 'diagonal']
neighbourhood = ['grid', 'euclidean']
windowsize = range(1, 7)
type = ['scatter', 'moran', 'spatstat']
buckets = range(30, 151, 30)
#bucket-lower-limit?
histnormalise = ('none', 'column', 'row')
k = range(2, 6)

In [ ]:
from sklearn import neighbors
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn import preprocessing

# Preprocessing (sharpening and background filtering)
def preprocess(integrals, sharpening, objectinfo, windowSize):
    sharpeningKernel = np.array([   [-1, -1,  -1],
                                    [-1,  9,  -1],
                                    [ -1, -1,  -1]
    ]) if sharpening == 'diagonal' else np.array([  [0, -1,  0],
                                                    [-1, 5, -1],
                                                    [0, -1,  0]])
    sharpened = np.array([cv2.filter2D(src=image, ddepth=-1, kernel=sharpeningKernel) for image in integrals])
# Sharpening is done
    filtered = np.full(np.shape(integrals), np.nan)
    for index, object in objectinfo.sort_values(by=['VED']).iterrows():
        if type == 'scatter':
            xs = np.array(sharpened[:, object.ystart:object.yend, object.xstart:object.xend], copy=True, dtype=np.float32)
            sum = np.sum(xs, axis=0)
            limit = np.percentile(sum, emptyRatio)
            filter = sum >= limit
            (endLayer, _, _) = sharpened.shape
            filter = np.repeat([filter], endLayer, 0)
            xs[~filter] = np.nan
            filtered[:, object.ystart:object.yend, object.xstart:object.xend] = xs
        elif type == 'moran':
            xs = sharpened[:, object.ystart:object.yend, object.xstart:object.xend]
            sum = np.sum(xs, axis=0)
            limit = np.percentile(sum, emptyRatio)
            filter = sum >= limit
            (endLayer, _, _) = sharpened.shape
            filter = np.repeat([filter], endLayer, 0)
            avg = np.mean(xs, where=filter)
            stddev = np.std(xs, where=filter)
            xs = (xs - avg) / avg
            xs[~filter] = np.nan
            filtered[:, object.ystart:object.yend, object.xstart:object.xend] = xs
        elif type == 'spatstat':
            # Implement later
            print("nyi")
    return sharpened

def neighbourhood(sharpened, neighbourhood, windowSize):
    neighbourkernel = np.array(
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]
    )/9 if neighbourhood == 'grid' else np.array(
        [[1, 2,  1],
        [2, 4, 2],
        [1, 2,  1]])/16
    flatNeighbourhood = np.array([cv2.filter2D(src=layer, ddepth=-1, kernel=neighbourkernel) for layer in sharpened])
    (endLayer, _, _) = flatNeighbourhood.shape
    neighbourhoodValues = np.array([
        np.sum(flatNeighbourhood[layerIndex-windowSize:layerIndex], axis=0)/windowSize
        for layerIndex in range(windowSize, endLayer)
    ])
    return neighbourhoodValues

def encode(sharpened, objectinfo, neighbourhoodValues, windowSize, type, buckets, histnormalise):
    offset = windowSize // 2
    (endLayer, _, _) = sharpened.shape
    # 1 -> 0
    # 2 -> 1
    # 3 -> 1

    if type == 'spatstat':
        print("nyi")
    else:
        outliervalues = np.full(np.shape(neighbourhoodValues), np.nan)
        for index, object in objectinfo.iterrows():
            if object.Label == "KH":
                continue
            xs = sharpened[offset:endLayer-windowSize+offset+1, object.ystart:object.yend, object.xstart:object.xend]
            ys = neighbourhoodValues[:, object.ystart:object.yend, object.xstart:object.xend]
            filter = np.logical_and(np.isfinite(xs), np.isfinite(ys))
            line = np.polyfit(ys[filter].flatten(), xs[filter].flatten(), 1)
            p = np.poly1d(line)
            outliervalues[:, object.ystart:object.yend, object.xstart:object.xend] = xs - p(ys)
        minval = np.min(outliervalues)
        maxval = np.max(outliervalues)
        for index, object in objectinfo.iterrows():
            if object.Label == "KH":
                continue
            xs = neighbourhoodValues[:, object.ystart:object.yend, object.xstart:object.xend]
            filter = np.isfinite(xs)
            label = object.Label
            hist, _ = np.histogram(xs[filter], buckets=buckets, range=(minval, maxval))
            if init == False:
                X = np.array([hist])
                Y = np.array([0 if label == 'GOOD' else 1])
                init = True
            else:
                X = np.append(X, np.array([hist]), 0)
                Y = np.append(Y, np.array([0 if label == 'GOOD' else 1]), 0)
        
        if (histnormalise == 'column'):
            X = preprocessing.normalize(X, axis=0)
        elif (histnormalise == 'row'):
            X = preprocessing.normalize(X, axis=1)
    return X, Y

def classify(X, Y, type, buckets, histnormalise, k):
loo = LeaveOneOut()
for n_neighbors in range(2,5):
    for weights in ["uniform", "distance"]:
        print("Neighbours " + str(n_neighbors) + " weights " + weights)
        clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
        scores = cross_val_score(clf, X, Y, scoring="accuracy", cv=loo.split(X))
        print(scores)
        print("%0.2f accuracy" % (scores.mean()))
    
